*************************************************************
*            Data de recomendación para el Modelo           *
*************************************************************

In [1]:
# Importacion de Librerias
import pandas as pd
import ast
import utils
import re
import gzip

# Librerias para el modelo de recomendación por similitud del coseno
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [169]:
# se implementa la data para recomendación de una  lista de 5 juegos a partir de un juego
# Se cargan dos dataset que requieren para el modelo

df_reviews = pd.read_parquet('Dataset\df_user_reviews_final.parquet')
df_items = pd.read_parquet('Dataset\df_user_items_final.parquet')

# Comprobamos el data de reviews
df_reviews.head()

<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\d'
C:\Users\juank\AppData\Local\Temp\ipykernel_11276\2495874675.py:4: SyntaxWarning: invalid escape sequence '\d'
  df_reviews = pd.read_parquet('Dataset\df_user_reviews_final.parquet')
C:\Users\juank\AppData\Local\Temp\ipykernel_11276\2495874675.py:5: SyntaxWarning: invalid escape sequence '\d'
  df_items = pd.read_parquet('Dataset\df_user_items_final.parquet')


,user_id,user_url,reviews_item_id,reviews_helpful,reviews_recommend,reviews,reviews_date,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011-11-05,2
1,js41637,http://steamcommunity.com/id/js41637,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2014-06-24,2
3,doctr,http://steamcommunity.com/id/doctr,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...,2013-10-14,2
4,maplemage,http://steamcommunity.com/id/maplemage,211420,35 of 43 people (81%) found this review helpful,True,Git gud,2014-04-15,1
5,Wackky,http://steamcommunity.com/id/Wackky,249130,7 of 8 people (88%) found this review helpful,True,This game is Marvellous.,2014-05-05,0


In [170]:
# Comprobamos el data de items
df_items.head()

,item_id,item_name,playtime_forever,steam_id,items_count,user_id,user_url
0,10,Counter-Strike,6,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,Team Fortress Classic,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,7,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,40,Deathmatch Classic,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
4,50,Half-Life: Opposing Force,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...


In [171]:
""" sobre reviews se decide adicionar una columna rating que apartir de sentiment_analysis y reviews_recommend, entoce se determina la siguiente escala: 
        1 si el análisis de sentimiento es negativo ya sea que este recomendado o no 
        2 si el análisis de sentimiento es neutral y no es recomendado 
        3 si el análisis de sentimiento es neutral pero es recomendado 
        4 si el análisis de sentimiento es positivo y no es recomendado 
        5 si el análisis de sentimiento es positivo y es recomendado   
   A partir de la siguiente función, configuramos la escala"""


def rating(row):    
    if row["sentiment_analysis"] == 0 and not row["reviews"]:
        return 1
    elif row["sentiment_analysis"] == 0 and row["reviews"]:
        return 1
    elif row["sentiment_analysis"] == 1 and not row["reviews"]:
        return 2
    elif row["sentiment_analysis"] == 1 and row["reviews"]:
        return 3
    elif row["sentiment_analysis"] == 2 and not row["reviews"]:
        return 4
    elif row["sentiment_analysis"] == 2 and row["reviews"]:
        return 5
    else:
        return None
    
# creamos la nueva columna ratintg y ulizamos la función
df_reviews['rating'] = df_reviews.apply(rating, axis=1)

# Revisamos la data
df_reviews


,user_id,user_url,reviews_item_id,reviews_helpful,reviews_recommend,reviews,reviews_date,sentiment_analysis,rating
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011-11-05,2,5
1,js41637,http://steamcommunity.com/id/js41637,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2014-06-24,2,5
3,doctr,http://steamcommunity.com/id/doctr,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...,2013-10-14,2,5
4,maplemage,http://steamcommunity.com/id/maplemage,211420,35 of 43 people (81%) found this review helpful,True,Git gud,2014-04-15,1,3
5,Wackky,http://steamcommunity.com/id/Wackky,249130,7 of 8 people (88%) found this review helpful,True,This game is Marvellous.,2014-05-05,0,1
...,...,...,...,...,...,...,...,...,...
231901,76561198138691719,http://steamcommunity.com/profiles/76561198138...,332310,No ratings yet,True,Normally I would hardly play a lego based game...,2015-12-30,0,1
231919,SKELETRONPRIMEISOP,http://steamcommunity.com/id/SKELETRONPRIMEISOP,440,No ratings yet,True,TF2 is alot of fun and its really good but the...,2014-08-15,2,5
231921,76561198141079508,http://steamcommunity.com/profiles/76561198141...,304930,No ratings yet,True,Fun game with friends,2014-08-02,0,1
232047,ShadowYT100,http://steamcommunity.com/id/ShadowYT100,265630,No ratings yet,True,So Fun!! :D,2015-07-31,2,5


In [172]:
# Definimos dataframe para reservar las columnas user_id, reviews_item_id y rating
reserva = df_reviews[['user_id', 'reviews_item_id', 'rating']]
reserva.head()

,user_id,reviews_item_id,rating
0,76561197970982479,1250,5
1,js41637,251610,5
3,doctr,250320,5
4,maplemage,211420,3
5,Wackky,249130,1


In [173]:
# Definimos dataframe para reservar las columnas  item_id y los nombres de los juego
juegos = df_items[['item_id', 'item_name']]

# Eliminamos duplicados
juegos = juegos.drop_duplicates()
juegos

,item_id,item_name
0,10,Counter-Strike
1,20,Team Fortress Classic
2,30,Day of Defeat
3,40,Deathmatch Classic
4,50,Half-Life: Opposing Force
...,...,...
4913278,375450,NOBUNAGA'S AMBITION: Souzou SengokuRisshiden
5092969,353390,Alienware Steam Machine
5105048,354280,ChaosTower
5119454,433920,Aveyond 4: Shadow Of The Mist


In [174]:
# Unimos los nombres de los juegos al data de reserva
datos = reserva.merge(juegos, left_on="reviews_item_id", right_on="item_id", how='left')
datos

,user_id,reviews_item_id,rating,item_id,item_name
0,76561197970982479,1250,5,1250,Killing Floor
1,js41637,251610,5,251610,Barbie™ Dreamhouse Party™
2,doctr,250320,5,250320,The Wolf Among Us
3,maplemage,211420,3,211420,Dark Souls: Prepare to Die Edition
4,Wackky,249130,1,249130,LEGO® MARVEL Super Heroes
...,...,...,...,...,...
48904,76561198138691719,332310,1,332310,LEGO® Worlds
48905,SKELETRONPRIMEISOP,440,5,NaN,NaN
48906,76561198141079508,304930,1,304930,Unturned
48907,ShadowYT100,265630,5,265630,Fistful of Frags


In [175]:
# revisemos el data datos
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48909 entries, 0 to 48908
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   user_id          48909 non-null  object
 1   reviews_item_id  48909 non-null  object
 2   rating           48909 non-null  int64 
 3   item_id          43154 non-null  object
 4   item_name        43154 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.9+ MB


In [176]:
# se revisan los reviews sin nombres de juegos en df_items
sin_juegos = datos[datos.isnull().any(axis=1)]

# Se calculan la cantidad de juegos en esta situación
print(f"Hay un total de {len(sin_juegos['reviews_item_id'].unique())} juegos con reviews pero que no estan en df_reviews")
sin_juegos

Hay un total de 255 juegos con reviews pero que no estan en df_reviews


,user_id,reviews_item_id,rating,item_id,item_name
14,devvonst,440,5,NaN,NaN
21,76561198076909484,570,1,NaN,NaN
27,boydeer,440,3,NaN,NaN
32,jarrodtrim,440,5,NaN,NaN
33,76561198067936203,440,5,NaN,NaN
...,...,...,...,...,...
48865,76561198074111910,440,5,NaN,NaN
48867,Fadingblaze,440,5,NaN,NaN
48889,MagmaCIaw,440,5,NaN,NaN
48896,mrscruff30,440,1,NaN,NaN


In [177]:
# Se elimina de la data datos
# Se borran los nulos
datos = datos.dropna(subset=['item_id'])

# Se analiza la data
datos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43154 entries, 0 to 48908
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   user_id          43154 non-null  object
 1   reviews_item_id  43154 non-null  object
 2   rating           43154 non-null  int64 
 3   item_id          43154 non-null  object
 4   item_name        43154 non-null  object
dtypes: int64(1), object(4)
memory usage: 2.0+ MB


In [178]:
# Se convierte la etiqueta item_id  a formato númerico y crea un dataframe recomendacion con las columnas necesarias

datos['item_id'] = datos['item_id'].astype(int)
recomendacion = datos[['user_id','item_id' , 'item_name', 'rating']]
recomendacion.head()

C:\Users\juank\AppData\Local\Temp\ipykernel_11276\437164019.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos['item_id'] = datos['item_id'].astype(int)


,user_id,item_id,item_name,rating
0,76561197970982479,1250,Killing Floor,5
1,js41637,251610,Barbie™ Dreamhouse Party™,5
2,doctr,250320,The Wolf Among Us,5
3,maplemage,211420,Dark Souls: Prepare to Die Edition,3
4,Wackky,249130,LEGO® MARVEL Super Heroes,1


In [179]:
recomendacion.shape
recomendacion.info()
recomendacion.columns

<class 'pandas.core.frame.DataFrame'>
Index: 43154 entries, 0 to 48908
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    43154 non-null  object
 1   item_id    43154 non-null  int32 
 2   item_name  43154 non-null  object
 3   rating     43154 non-null  int64 
dtypes: int32(1), int64(1), object(2)
memory usage: 1.5+ MB


Index(['user_id', 'item_id', 'item_name', 'rating'], dtype='object')

In [180]:
# Se finaliza guardando el dataframe recomendación en formato comprimido en parquet
recomendacion.to_parquet('dataset/df_recomendacion.parquet', engine='pyarrow')

In [181]:
# Reservamos el dataframe recomdacion y lo copiamos a un nuevo dataframe llamado data_modelo

data_modelo = recomendacion.copy()
#data_modelo = data_modelo.sample(n=25000, random_state=123)
data_modelo

,user_id,item_id,item_name,rating
0,76561197970982479,1250,Killing Floor,5
1,js41637,251610,Barbie™ Dreamhouse Party™,5
2,doctr,250320,The Wolf Among Us,5
3,maplemage,211420,Dark Souls: Prepare to Die Edition,3
4,Wackky,249130,LEGO® MARVEL Super Heroes,1
...,...,...,...,...
48903,76561198133319761,238460,BattleBlock Theater,5
48904,76561198138691719,332310,LEGO® Worlds,1
48906,76561198141079508,304930,Unturned,1
48907,ShadowYT100,265630,Fistful of Frags,5


In [182]:
# Verificar si en la columna user_id hay nulos
is_numeric = pd.to_numeric(data_modelo['user_id'], errors='coerce')
print(is_numeric)

0        7.656120e+16
1                 NaN
2                 NaN
3                 NaN
4                 NaN
             ...     
48903    7.656120e+16
48904    7.656120e+16
48906    7.656120e+16
48907             NaN
48908             NaN
Name: user_id, Length: 43154, dtype: float64


In [183]:
# Eliminar las filas con valores NaN en 'user_id'
data_modelo = data_modelo.dropna(subset=['user_id'])

In [184]:
# Verificar si todos los valores de 'rating' son numéricos
is_numeric = pd.to_numeric(data_modelo['rating'], errors='coerce')
print(is_numeric)

0        5
1        5
2        5
3        3
4        1
        ..
48903    5
48904    1
48906    1
48907    5
48908    5
Name: rating, Length: 43154, dtype: int64


In [185]:
# Mostrar las filas que tienen valores no numéricos en 'rating'
non_numeric_rows = data_modelo[is_numeric.isna()]
print("Filas con valores no numéricos en 'rating':")
print(non_numeric_rows)

Filas con valores no numéricos en 'rating':
Empty DataFrame
Columns: [user_id, item_id, item_name, rating]
Index: []


In [186]:
# Convertir la columna 'rating' a numérico (esto convertirá valores no numéricos a NaN)
data_modelo['rating'] = pd.to_numeric(data_modelo['rating'], errors='coerce')

In [187]:
# Verificar si todos los valores de 'rating' son numéricos y convertir
data_modelo['rating'] = pd.to_numeric(data_modelo['rating'], errors='coerce')
data_modelo['user_id'] = pd.to_numeric(data_modelo['user_id'], errors='coerce')

In [188]:
if data_modelo['rating'].isna().any():
    print("Hay valores no numéricos en 'rating' que se han convertido a NaN.")
    # Opcional: Puedes decidir qué hacer con estos valores NaN, por ejemplo, eliminarlos:
    data_modelo = data_modelo.dropna(subset=['rating'])
else:
    print("Todos los valores en 'rating' son numéricos.")

print(data_modelo)

Todos los valores en 'rating' son numéricos.
            user_id  item_id                           item_name  rating
0      7.656120e+16     1250                       Killing Floor       5
1               NaN   251610           Barbie™ Dreamhouse Party™       5
2               NaN   250320                   The Wolf Among Us       5
3               NaN   211420  Dark Souls: Prepare to Die Edition       3
4               NaN   249130           LEGO® MARVEL Super Heroes       1
...             ...      ...                                 ...     ...
48903  7.656120e+16   238460                 BattleBlock Theater       5
48904  7.656120e+16   332310                        LEGO® Worlds       1
48906  7.656120e+16   304930                            Unturned       1
48907           NaN   265630                    Fistful of Frags       5
48908           NaN   304050                               Trove       5

[43154 rows x 4 columns]


In [189]:
# Verificar si todos los valores de 'rating' son numéricos y convertir
data_modelo['rating'] = pd.to_numeric(data_modelo['rating'], errors='coerce')
data_modelo['user_id'] = pd.to_numeric(data_modelo['user_id'], errors='coerce')

In [190]:
# Eliminar las filas con valores NaN en cualquier columna
data_modelo = data_modelo.dropna()

In [134]:
nan_count = data_modelo['rating'].isna().sum()

# Mostrar el resultado
print(f"Número de valores NaN en la columna 'features': {nan_count}")

Número de valores NaN en la columna 'features': 0


In [196]:
result = data_modelo[data_modelo['item_id'] == 72850]
print(result.iloc[0]['item_name'])

The Elder Scrolls V: Skyrim


In [216]:
data_modelo

,user_id,item_id,item_name,rating
0,7.656120e+16,1250,Killing Floor,5
5,7.656120e+16,72850,The Elder Scrolls V: Skyrim,1
11,7.656120e+16,33440,Driver San Francisco,1
16,7.656120e+16,8870,BioShock Infinite,3
17,7.656120e+16,368230,Kingdom: Classic,1
...,...,...,...,...
48585,7.656120e+16,275490,Canyon Capers,3
48603,7.656120e+16,259550,Hero of the Kingdom,1
48634,7.656120e+16,305960,Dragon: The Game,5
48774,7.656120e+16,340150,Basement,5


In [214]:
data_modelo['item_name'].value_counts()

item_name
Killing Floor                      1
Football Manager 2014              1
Nosferatu: The Wrath of Malachi    1
Call of Duty: Black Ops - OSX      1
The Impossible Game                1
                                  ..
METAL GEAR RISING: REVENGEANCE     1
Crysis                             1
Surgeon Simulator                  1
Galaxy on Fire 2™ Full HD          1
Trials 2: Second Edition           1
Name: count, Length: 1768, dtype: int64

In [213]:
data_modelo.drop_duplicates(subset='item_name', inplace=True)

In [215]:
# Se finaliza guardando el dataframe recomendación en formato comprimido en parquet
data_modelo.to_parquet('dataset/df_modelo_similitud.parquet', engine='pyarrow')

In [200]:
# Primero, vamos a convertir la columna 'item_name' a una representación numérica usando TF-IDF (Frecuencia de Terminos - Freciencia Inversa de Terminos)
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(data_modelo['item_name'])

# Luego, vamos a añadir las columnas numéricas a nuestra matriz de características
features = np.column_stack([tfidf_matrix.toarray(), data_modelo['rating'], data_modelo['user_id'],])

In [3]:
""" # Verificamos la existencia del Id del juego a establecer simulitud de juegos
result = data_modelo[data_modelo['item_id'] == 238460]
juego_nombre=result.iloc[0]['item_name']

# Reindexamos el DataFrame
data_aplicativo = data_modelo.reset_index(drop=True)

# Ahora, calculamos la matriz de similitud de coseno
similarity_matrix = cosine_similarity(features)

# Para hacer recomendaciones, puedes buscar los juegos más similares a un juego dado
nombre_del_juego = juego_nombre  #"LEGO® MARVEL Super Heroes"  
juego = data_aplicativo[data_aplicativo['item_name'] == nombre_del_juego]

if not juego.empty:
    product_index = juego.index[0]
    product_similarities = similarity_matrix[product_index]
    most_similar_products_indices = np.argsort(-product_similarities)[:6]
    most_similar_products = data_aplicativo.loc[most_similar_products_indices, 'item_name']
    print("Los juegos más similares al juego", nombre_del_juego, "son:")
    print(most_similar_products)
else:
    print("Juego no encontrado") """



 #Cargar el dataframe para aplicar el modelo
data_modelo = pd.read_parquet('Dataset/df_modelo_similitud.parquet') 

id=368230

# Primero, vamos a convertir la columna 'item_name' a una representación numérica usando TF-IDF (Frecuencia de Terminos - Freciencia Inversa de Terminos)
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(data_modelo['item_name'])

# Luego, vamos a añadir las columnas numéricas a nuestra matriz de características
features = np.column_stack([tfidf_matrix.toarray(), data_modelo['rating']])

# Verificamos la existencia del Id del juego a establecer simulitud de juegos
result = data_modelo[data_modelo['item_id'] == id]
nombre_del_juego=result.iloc[0]['item_name']

# Reindexamos el DataFrame
data_aplicativo = data_modelo.reset_index(drop=True)

# Ahora, calculamos la matriz de similitud de coseno
similarity_matrix = cosine_similarity(features)

# Para hacer recomendaciones, puedes buscar los juegos más similares a un juego dado
juego = data_aplicativo[data_aplicativo['item_id'] == id].index[0]
score = list(enumerate(similarity_matrix[juego]))
score= sorted(score, key=lambda x: x[1], reverse=True)
resultado = score[1:6]
total = data_modelo['item_name'].iloc[[i[0] for i in resultado]].tolist()
print({'Juego Recomendado ': total})


{'Juego Recomendado ': ['Kingdom Wars', 'Kingdom Rush', 'Aura Kingdom', 'Team Fortress Classic', 'Divinity: Original Sin (Classic)']}
